<a href="https://colab.research.google.com/github/Nohyunjin/DeepLearning/blob/main/bbc%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import numpy as np
from time import time

import nltk

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Embedding
from keras.layers import Bidirectional

In [2]:
MY_WORDS = 5000
MY_EMBED = 64
MY_HIDDEN = 100
MY_LEN = 200

MY_SPLIT = 0.8
MY_SAMPLE = 123
MY_EPOCH = 10

In [19]:
original = []
processed = []
labels = []

In [20]:
nltk.download("stopwords")
MY_STOP = set(nltk.corpus.stopwords.words('english'))

print("불용어 수:", len(MY_STOP))
print(MY_STOP)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
불용어 수: 179
{'is', 'be', 'themselves', 'under', 'all', 'am', 'won', 'how', 'mightn', 'same', 'own', 'from', 'more', 'up', 'before', 'you', 'doing', 'that', 'because', 'above', "it's", 'over', 'against', 'yourself', 'don', 'shouldn', 'now', 'both', 'his', 'other', 'only', 'couldn', "hadn't", 'mustn', 'so', 'no', 'needn', 'will', 'during', "she's", "wouldn't", 'has', 'those', 'too', 's', 'there', 'had', 'why', 'an', 'o', 'can', 'weren', 'while', "hasn't", 'it', 'on', 'and', 'here', 'been', 'few', 'again', "should've", 'ma', 'for', 'to', 'we', "isn't", 'each', 'very', "shan't", 'll', 'himself', 're', 'do', 'after', "you'll", 'ours', "doesn't", 'didn', 'its', 'in', 'doesn', 'your', 'wouldn', 'our', 'not', 'aren', "didn't", 'being', 'than', 'my', 'did', 'any', 'their', 'them', 'once', 'if', 'through', 'i', 'haven', 'which', 'd', 'shan', 'but', 'me', 'by', 'of', 've', 'theirs

In [21]:
with open('bbc-text.csv', 'r') as file :
    reader = csv.reader(file)
    next(reader)
    for row in reader :
        labels.append(row[0]) # 라벨
        original.append(row[1]) # 원본 텍스트
        news = row[1]
        for word in MY_STOP:
            token = ' ' + word + ' '
            news = news.replace(token, ' ')
        processed.append(news) # 불용어가 삭제된 텍스트

In [22]:
sample = original[MY_SAMPLE]
print('불용어 삭제 전 샘플 기사')
print(sample)
print('카테고리:', labels[MY_SAMPLE])
print('단어 수:', len(sample.split()))

불용어 삭제 전 샘플 기사
screensaver tackles spam websites net users are getting the chance to fight back against spam websites  internet portal lycos has made a screensaver that endlessly requests data from sites that sell the goods and services mentioned in spam e-mail. lycos hopes it will make the monthly bandwidth bills of spammers soar by keeping their servers running flat out. the net firm estimates that if enough people sign up and download the tool  spammers could end up paying to send out terabytes of data.   we ve never really solved the big problem of spam which is that its so damn cheap and easy to do   said malte pollmann  spokesman for lycos europe.  in the past we have built up the spam filtering systems for our users   he said   but now we are going to go one step further.    we ve found a way to make it much higher cost for spammers by putting a load on their servers.  by getting thousands of people to download and use the screensaver  lycos hopes to get spamming websites consta

In [23]:
print('불용어 삭제 후 샘플 기사')
sample = processed[MY_SAMPLE]
print(sample)
print('단어 수:', len(sample.split()))

불용어 삭제 후 샘플 기사
screensaver tackles spam websites net users getting chance fight back spam websites  internet portal lycos made screensaver endlessly requests data sites sell goods services mentioned spam e-mail. lycos hopes make monthly bandwidth bills spammers soar keeping servers running flat out. net firm estimates enough people sign download tool  spammers could end paying send terabytes data.   never really solved big problem spam damn cheap easy   said malte pollmann  spokesman lycos europe.  past built spam filtering systems users   said   going go one step further.    found way make much higher cost spammers putting load servers.  getting thousands people download use screensaver  lycos hopes get spamming websites constantly running almost full capacity. mr pollmann said intention stop spam websites working subjecting much data cope with. said screensaver carefully written ensure amount traffic generated user overload web.  every single user contribute three four megabytes per 

In [24]:
A_token = Tokenizer(num_words = MY_WORDS, oov_token = 'OOV')

A_token.fit_on_texts(processed)
A_tokenized = A_token.texts_to_sequences(processed)

maxlen = max([len(x) for x in A_tokenized])
minlen = min([len(x) for x in A_tokenized])

print('토큰화 결과')
print(f'총 단어 수: {len(A_token.word_counts)}')
print(f'가장 긴 기사: {maxlen}')
print(f'가장 짧은 기사: {minlen}')

토큰화 결과
총 단어 수: 29698
가장 긴 기사: 2279
가장 짧은 기사: 50


In [32]:
A_tokenized = pad_sequences(A_tokenized, maxlen = MY_LEN, padding = 'post', truncating = 'post')
A_tokenized = np.array(A_tokenized)

sample = A_tokenized[MY_SAMPLE]
print('샘플 기사 토큰화 후 패딩 결과')
print(sample)
print('단어 수:', len(sample))
print('샘플 shape:', sample.shape)

샘플 기사 토큰화 후 패딩 결과
[3171    1  816  877  115  136  382  347  716   28  816  877  228    1
 3172   27 3171    1 4869  203  568  733 1770  126 4025  816  260  395
 3172  700   21 1648 3629 2849 2607    1 2325 2551  453 2919  569  115
   63 2291  381    7 1161  780 1859 2607   11   92 1570 1051    1  203
  281  154    1  138  364  816    1 2224  847    2    1    1  178 3172
  139  255 1110  816    1  726  136    2   52   60   10  818 3790  195
   41   21   56  494  245 2607 1363    1 2551  382 1021    7  780   70
 3171 3172  700   23    1  877 3993  453  343  322 1394    3    1    2
 3428  582  816  877  297    1   56  203 2296 2357    2 3171 2709 1069
  660  812 1287 3885 1538    1  466  224  503 1538    1   31   96    1
  681  111    2   10 1897  912    2  381    7 1161    1  877   11  722
  256    1 1287  224  503  111 3172   79   70  260  395  716   28    2
    3    1    4 1604   10  823  455  158  823  455    2  568 2178 4025
  816  260  395  891  733 1770  126  220 3677  568  316   8

In [31]:
C_token = Tokenizer()
C_token.fit_on_texts(labels)
C_tokenized = C_token.texts_to_sequences(labels)
C_tokenized = np.array(C_tokenized).reshape(-1)

print('카테고리 토큰화 결과')
print(C_token.word_index)
print('샘플 기사 카테고리:', C_tokenized[MY_SAMPLE])
print('토큰화 결과 shape:', C_tokenized.shape)

카테고리 토큰화 결과
{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
샘플 기사 카테고리: 4
토큰화 결과 shape: (2225,)


In [29]:
idx_to_label = {}
for label, index in C_token.word_index.items():
    idx_to_label[index] = label

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(A_tokenized, C_tokenized, train_size = MY_SPLIT, shuffle = False)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1780, 200) (1780,) (445, 200) (445,)


In [33]:
model = Sequential()
model.add(Embedding(input_dim = MY_WORDS, output_dim = MY_EMBED))
model.add(Dropout(rate = 0.5))
model.add(Bidirectional(LSTM(units = MY_HIDDEN)))
model.add(Dense(units = 6, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 200)              132000    
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 1206      
                                                                 
Total params: 453,206
Trainable params: 453,206
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [36]:
begin = time()
model.fit(x=X_train, y = Y_train, epochs = MY_EPOCH, verbose = 2)
end = time()
print(f'학습시간: {end-begin:.2f}')

Epoch 1/10
56/56 - 6s - loss: 1.5939 - acc: 0.2708 - 6s/epoch - 116ms/step
Epoch 2/10
56/56 - 1s - loss: 1.2381 - acc: 0.4904 - 1s/epoch - 18ms/step
Epoch 3/10
56/56 - 1s - loss: 0.5584 - acc: 0.8067 - 1s/epoch - 19ms/step
Epoch 4/10
56/56 - 1s - loss: 0.5187 - acc: 0.8427 - 1s/epoch - 19ms/step
Epoch 5/10
56/56 - 1s - loss: 0.1979 - acc: 0.9562 - 1s/epoch - 19ms/step
Epoch 6/10
56/56 - 1s - loss: 0.1010 - acc: 0.9764 - 1s/epoch - 19ms/step
Epoch 7/10
56/56 - 1s - loss: 0.0444 - acc: 0.9933 - 1s/epoch - 19ms/step
Epoch 8/10
56/56 - 1s - loss: 0.0467 - acc: 0.9904 - 1s/epoch - 19ms/step
Epoch 9/10
56/56 - 1s - loss: 0.0140 - acc: 0.9989 - 1s/epoch - 19ms/step
Epoch 10/10
56/56 - 1s - loss: 0.0062 - acc: 1.0000 - 1s/epoch - 19ms/step
학습시간: 16.14


In [37]:
score = model.evaluate(x = X_test, y = Y_test, verbose = 2)
print(f'최종 정확도: {score[1]}')

14/14 - 1s - loss: 0.1539 - acc: 0.9618 - 800ms/epoch - 57ms/step
최종 정확도: 0.9617977738380432


In [38]:
pred = model.predict(x = X_test)
pred = np.argmax(pred, axis = 1)
print(pred)

[5 4 3 1 1 4 2 4 5 5 3 3 2 5 1 5 5 2 1 3 4 2 1 5 4 3 3 1 1 3 2 2 2 2 5 2 3
 3 4 4 5 3 5 2 3 1 1 2 4 2 4 1 2 2 3 1 1 3 2 5 5 3 2 2 3 2 4 2 3 3 3 3 5 5
 4 3 1 1 1 4 5 1 1 5 4 5 4 1 4 1 1 5 5 2 5 5 3 2 1 4 4 3 2 1 2 5 1 3 5 1 1
 2 3 4 4 2 2 1 3 5 1 1 3 5 4 1 5 2 3 1 3 4 5 1 3 2 5 3 5 3 1 3 2 2 3 2 4 1
 2 5 2 1 1 5 4 3 4 3 3 1 1 1 2 4 5 2 1 2 1 2 4 2 2 2 2 1 1 1 2 2 5 2 2 2 1
 1 1 4 2 1 1 1 2 5 4 4 4 3 2 2 4 2 4 1 1 3 3 3 1 1 3 3 4 2 1 1 1 1 2 1 2 2
 2 2 1 3 1 4 4 1 4 2 5 2 1 2 4 4 3 5 2 5 2 4 3 5 2 5 5 4 2 4 4 2 3 1 5 2 3
 5 2 4 1 4 3 1 3 2 3 3 2 2 2 4 3 2 3 2 5 3 1 3 3 1 5 4 4 2 4 1 2 2 2 1 4 4
 4 1 5 1 3 2 3 3 5 4 2 4 1 5 5 1 2 5 4 4 1 5 2 3 3 3 4 4 2 3 2 4 3 5 5 2 2
 4 5 4 4 1 3 1 1 3 5 5 2 3 3 1 2 2 4 2 4 4 1 2 3 1 2 2 1 4 1 4 5 1 1 5 2 4
 1 1 3 4 2 3 1 1 3 2 4 4 2 2 1 5 5 4 2 3 4 1 1 4 4 3 2 1 5 5 1 3 4 1 2 2 2
 1 1 4 1 2 4 2 2 1 2 3 2 2 5 3 4 3 4 5 3 4 5 1 3 5 2 4 2 4 5 4 1 2 2 3 5 3
 1]


In [39]:
print(confusion_matrix(y_true = Y_test, y_pred = pred))

[[ 99   0   0   0   2]
 [  0 104   2   0   0]
 [  1   5  79   1   0]
 [  1   2   0  82   1]
 [  0   1   1   0  64]]


In [40]:
import re
news = ['''
The International Space Station (ISS) will continue working until 2030, before plunging into the Pacific Ocean in early 2031, according to Nasa.

In a report this week, the US space agency said the ISS would crash into a part of the ocean known as Point Nemo.

This is the point furthest from land on planet Earth, also known as the spacecraft cemetery.

Many old satellites and other space debris have crashed there, including the Russian space station Mir in 2001.

Nasa said that in the future space activities close to Earth would be led by the commercial sector.

The ISS - a joint project involving five space agencies - has been in orbit since 1998 and has been continuously crewed since 2000. More than 3,000 research investigations have taken place in its microgravity laboratory.
'''.lower()]

news[0] = re.sub(r'[^\w\s]','', news[0])

print("불용어 처리 전 문자 수:", len(news[0]))
for word in MY_STOP:
    token = ' ' + word + ' '
    news[0] = news[0].replace(token, ' ')
print("불용어 처리 후 문자 수:", len(news[0]))

seq = A_token.texts_to_sequences(news)
padded = pad_sequences(seq, maxlen = MY_LEN, padding = 'post', truncating = 'post')

my_pred = model.predict(padded)
print("예측 결과:", idx_to_label[np.argmax(my_pred[0])])


불용어 처리 전 문자 수: 774
불용어 처리 후 문자 수: 585
예측 결과: tech
